In [5]:
from matplotlib import pyplot as plt
import numpy as np
import csv
import heapq

input_file = "traces.csv"

In [6]:
# Read the CSV file
with open(input_file, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    # Skip the header
    next(reader)
    # Read the data into memory
    data = []
    for row in reader:
        # convert row to ints
        row = [int(i) for i in row]
        data.append(row)

In [33]:
# Question 1

def track_peaks (trace):
    peak_minimum = 10000
    peak_window = 20
    history = []
    peak_indices = []
    found_peak_in_previous_window = False
    cooling_off = False
    max_tuple = (0, 0)
    for i in range(0, len(trace)):
        if len(history) < peak_window - 1:
            history.append(trace[i])
        else:
            history.append(trace[i])

            window_max = (max(history), history.index(max(history)) + i - peak_window + 1)
            peak_high_enough = window_max[0] > peak_minimum
            if window_max[0] < max_tuple[0] and found_peak_in_previous_window:
                peak_indices.append(max_tuple[1])
                found_peak_in_previous_window = False
                max_tuple = (0, 0)
                cooling_off = True
            elif cooling_off:
                if not peak_high_enough:
                    cooling_off = False
            elif peak_high_enough:
                max_tuple = window_max
                found_peak_in_previous_window = True

            # print(i, trace[i], window_max, max_tuple, found_peak_in_previous_window)
            history.pop(0)

    return peak_indices

# plt.plot(data[3])
# print(data[3])
# print(track_peaks(data[3]))


all_peaks = []
for i in range(0, len(data)):
    all_peaks.append(track_peaks(data[i]))
    num_peaks = len(all_peaks[i])

